In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df1 = pd.read_csv('triplets_file.csv')
print(len(df1))
df1.head()


1048575


,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [3]:
df2 = pd.read_csv('song_data_rec.csv')
print(len(df2))
df2.head()


689198


,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [4]:
df_merged= pd.merge( df2,df1, on='song_id', how='right')
df_merged.head()


,song_id,title,release,artist_name,year,user_id,listen_count
0,SOAKIMP12A8C130995,The Cove,Thicker Than Water,Jack Johnson,0.0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,SOBBMDR12A8C13253B,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2
2,SOBXHDL12A81C204C0,NaN,NaN,NaN,NaN,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,SOBYHAJ12A6701BF1D,NaN,NaN,NaN,NaN,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,SODACBL12A8C13C273,NaN,NaN,NaN,NaN,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


# Recommending song based on popularity

In [5]:
df3=df_merged.groupby('title').sum()
df3=df3. sort_values(by=["listen_count"],ascending=False)

df3.drop('year',inplace = True, axis = 1)
df3=df3.reset_index()
df3.index = df3.index + 1
print("Top 10 songs based on Popularity: ")
df3.head(10)

Top 10 songs based on Popularity: 


,title,listen_count
1,Horn Concerto No. 4 in E flat K495: II. Romanc...,16277
2,Sehr kosmisch,15813
3,Dog Days Are Over (Radio Edit),14031
4,Ain't Misbehavin,9931
5,Représente,9590
6,Catch You Baby (Steve Pitron & Max Sanna Radio...,9434
7,Sincerité Et Jalousie,8786
8,Hey_ Soul Sister,8027
9,Tive Sim,7410
10,Make Love To Your Mind,5600


# Recommending Songs Based on Similarity with other Song

In [26]:
#taking the trimmed dataset
df=df2.head(3000)



In [27]:
#creating important features
def get_important_features(data):
    important_features = []
    for i in range(0, data.shape[0]):
        important_features.append(str(data['title'][i])+' '+str(data['release'][i])+' '+str(data['artist_name'][i])+' '+str(data['year'][i]))
    return important_features


#create a column to hold the combine strings
df['important_features'] = get_important_features(df)




#convert the text to a matrix of token counts
cm = CountVectorizer().fit_transform(df['important_features'])

#get the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)

#get the shape of cosine similarity matrix
print(cs.shape)

df.head(5)






<ipython-input-27-78d91c8cd73f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['important_features'] = get_important_features(df)


(3000, 3000)


,song_id,title,release,artist_name,year,important_features
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,Silent Night Monster Ballads X-Mas Faster Puss...
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,Tanssi vaan Karkuteillä Karkkiautomaatti 1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,No One Could Ever Butter Hudson Mohawke 2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,Si Vos Querés De Culo Yerba Brava 2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,Tangle Of Aspens Rene Ablaze Presents Winter S...


In [28]:
df = df.reset_index(drop = True)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()


def get_recommendations(title, cs=cs):
    # Get the index of the song that matches the title
    idx = indices[title]
    
    # Get the pairwsie similarity scores of all songswith that songs
    sim_scores = list(enumerate(cs[idx]))
    
            
   
    
    # Sort the songs based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse = True)
   
   
    # Get the scores of the 10 most similar songs
    sim_scores = sim_scores[1:11]
    
    # Get the song indices
    song_indices = [i[0] for i in sim_scores]
    
    
    # Return the top 10 most similar songs
    return df['title'].iloc[song_indices]

In [29]:
get_recommendations('Silent Night')

2665                         Faster Faster
1265               Monster (Album Version)
1172                    In The Summer Camp
1932                   Las Calles de Altea
1327    Over On 4th Street (Album Version)
2447               The Night He Got Caught
2292                            Body Thief
2826               This Is War (Rough Mix)
72                   The Rest of the Night
1930                        Vampiria Night
Name: title, dtype: object

In [30]:
get_recommendations('Goodbye')

1142          Perfect Goodbye
2962       Boy With The Blues
766      Islands (LP Version)
1458        Scream Atomic Boy
1513             Goodbye Love
1573            I Got A Stick
2089                    Paris
2659        Astro Boy (Album)
560              Just a Dream
2222    Scar That Never Heals
Name: title, dtype: object